**Loading the API Key and relevant python librariers**

In [2]:
import os
import getpass
from google import genai

if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass.getpass("Paste your Gemini API Key: ")

client = genai.Client()

Paste your Gemini API Key:  ········


**Defining helper function to make it easier to use prompts and look at generated outputs using gpt-3.5-turbo**

In [3]:
def get_completion(prompt, model="gemini-2.5-flash-lite"):
    # Request generation from Gemini
    response = client.models.generate_content(
        model=model,
        contents=prompt
    )
    return response.text

**Tactic 1: Use delimiters to clearly indicate distinct parts of the input
Delimiters can be anything like: ```, """, < >, <tag> </tag>, :**

In [9]:
text = """
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. This will \ 
guide the model towards the desired output...
"""

prompt = f"Summarize the text delimited by triple backticks into a single sentence. ```{text}```"

# Get and print the response
try:
    response = get_completion(prompt)
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")

To guide a model towards a desired output, you should provide clear and specific instructions.


**Tactic 2: Ask for a structured output**

In [10]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": "BK001",
    "title": "The Lumina Obscura",
    "author": "Elias Thorne",
    "genre": "Gothic Fantasy"
  },
  {
    "book_id": "BK002",
    "title": "Chronos Gambit",
    "author": "Seraphina Vance",
    "genre": "Sci-Fi Thriller"
  },
  {
    "book_id": "BK003",
    "title": "Whispers of the Amber Coast",
    "author": "Isolde Finch",
    "genre": "Historical Romance"
  }
]
```


**Tactic 3: Ask the model to check whether conditions are satisfied**

In [4]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.


In [5]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


**Tactic 4: "Few-shot" prompting**

In [6]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The oak that withstands the fiercest storm has roots that grow deep into the earth; the mountain that stands unyielding to the wind is shaped by eons of pressure; the phoenix that rises from its ashes is reborn through enduring flames.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [7]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Siblings Jack and Jill embarked on a water-fetching adventure that ended with a tumble down a hill, but their spirits remained undimmed.

Les frères et sœurs Jack et Jill se sont lancés dans une aventure pour aller chercher de l'eau qui s'est terminée par une chute dans une colline, mais leur esprit est resté intact.

Jack, Jill

```json
{
  "french_summary": "Les frères et sœurs Jack et Jill se sont lancés dans une aventure pour aller chercher de l'eau qui s'est terminée par une chute dans une colline, mais leur esprit est resté intact.",
  "num_names": 2
}
```


**Ask for output in a specified format**

In [8]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Siblings Jack and Jill's quest for water ends in a tumble down the hill, but their adventurous spirits remain undimmed.
Translation: La quête d'eau des frères et sœurs Jack et Jill se termine par une chute en bas de la colline, mais leurs esprits aventureux restent intacts.
Names: Jack, Jill
Output JSON:
```json
{
  "french_summary": "La quête d'eau des frères et sœurs Jack et Jill se termine par une chute en bas de la colline, mais leurs esprits aventureux restent intacts.",
  "num_names": 2
}
```
